# Variables categóricas

En esta clase vamos a ver como utilizar pandas y scikit learn para transformar variables categóricas en algo que los modelos de machine learning puedan entender.

Vamos a utilizar un dataset armado a mano y bastante simple para aprender a utilizar scikit learn y pandas.

Luego, tendrán que aplicar lo aprendido sobre el dataset de la clase pasada (ecommerce).

In [ ]:
#from google.colab import drive # La usamos para montar nuestra unidad de Google Drive
#drive.mount('/content/drive') # Montamos nuestra unidad de Google Drive

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data = {'Temperature': ['Hot', 'Cold', 'Very Hot', 'Warm', 'Hot', 'Warm', 'Warm', 'Hot', 'Hot', 'Cold'],
        'Color': ['Red', 'Yellow','Blue', 'Blue', 'Red', 'Yellow', 'Red', 'Yellow', 'Yellow', 'Blue']}

df = pd.DataFrame(data)

In [ ]:
df.Color.nunique()

## One hot encoding

En este simple caso, vemos que la variable Temperature puede ser considerada ordinal porque la temperatura va desde cold hasta very hot.

Por otro lado, en la variable color no vemos ningún orden, no podemos considerarla ordinal.

Vamos a aplicar one hot encoding en la variable color.

Esto se puede hacer con pandas o con el OneHotEncoder de scikit learn.

Comencemos con pandas.

Pandas nos brinda la funcion get_dummies():

In [ ]:
pd.get_dummies(df.Color)

¿ Cómo agregamos estas columnas a nuestro dataset ?

Podemos concatenar horizontalmente este dataset de variables dummies a el original:

La próxima clase veremos en más detalle los métodos concat, merge, etcétera.

In [ ]:
dummies = pd.get_dummies(df.Color) # Obtenemos dummies
df_encoded = pd.concat([df, dummies], axis=1) # Concatenamos horizontalmente con axis=1

df_encoded

Ahora podemos eliminar la columna original

In [ ]:
df_encoded = df_encoded.drop('Color', axis=1)

In [ ]:
df_encoded

¿ Cómo hacemos lo mismo con scikit learn ?

Tenemos el OneHotEncoder en el módulo de preprocessing:

https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html#sklearn.preprocessing.OneHotEncoder



In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)

In [ ]:
# handle_unknow: si se encuentra con una categoria desconocida entonces completa todas las columnas con 0's
# sparse matrix ??? que nos dice? para que se usa?
# sparse bool, default=True Will return sparse matrix if set True else will return an array.

Averiguar en la documentación: 
- ¿ Qué significa el `handle_unknown='ignore'` ?
- Que es "sparse" ?
- Que sucede si ponemos sparse = True?

Hacemos fit:

In [ ]:
enc.fit(df.Color.values.reshape(-1,1))

In [ ]:
df.Color.values.reshape(-1,1)

- ¿ Qué pasa si sacamos el .reshape(-1, 1) ? -----> se rompe

- ¿ Qué otra forma se les ocurre para solucionar el error sin usar reshape ?

In [ ]:
encoded_color = enc.transform(df.Color.values.reshape(-1, 1))
encoded_color

Ahora, como agregamos esto a nuestro dataframe?

El método get_feature_names nos da los nombres de las nuevas features creadas:

In [ ]:
enc.get_feature_names(['Colores'])

In [ ]:
encoded_color_columns = enc.get_feature_names(['Colores'])

In [ ]:
encoded_color_df = pd.DataFrame(data=encoded_color, columns= encoded_color_columns)
encoded_color_df

Ahora, como hicimos antes, podemos concatenar y eliminar la columna original:

In [ ]:
pd.concat([df, encoded_color_df], axis=1).drop('Color', axis=1)

Muchas veces, en lugar de crear todas las columnas, se utiliza el atributo `drop='first'`.

Esto crea todas las columnas menos la primera (en nuestro caso no se crearía color_blue), esto es porque si ninguna de las otras es 1, significa que blue es 1. Sirve para ahorrarnos una columna.

En el caso de variables binarias, podemos crear una única columna utilizando:

`drop='if_binary'`

## Label encoder

Se utiliza de una forma muy similar a el OneHotEncoder de scikit learn.

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder() #creo el objeto

df['Temperature_label_encoded'] = le.fit_transform(df.Temperature)
df

No se utiliza para datos ordinales ya que scikit learn le asigna un valor numérico pero sin tener en cuenta que nosotros queremos que cold sea menor que hot.

Cuando queremos especificar nosotros los valores numéricos para cada valor de la variable categórica, podemos utilizar la función .replace() de pandas.

Esta función recibe un diccionario en el que la key tiene que ser el valor que queremos transformar y el value el valor resultante que queremos.

Veamos un ejemplo:

In [ ]:
df.Temperature.unique()

In [ ]:
mapping_dict = {
    'Cold': 1,
    'Warm': 2,
    'Hot': 3,
    'Very Hot': 4
}

temperature_ordinal = df.Temperature.replace(mapping_dict)
temperature_ordinal

In [ ]:
df['Temperature_ordinal'] = temperature_ordinal
df

In [ ]:
df.drop('Temperature_label_encoded', axis=1, inplace=True)

In [ ]:
df

# Discretización

Vamos a ver como hacerlo con sklearn. Para este caso vamos a utilizar otro dataset con una variable continua:

Creamos el dataset:

In [ ]:
variable_continua = np.arange(200)
df_cont = pd.DataFrame({'X': variable_continua})

In [ ]:
df_cont.head()

Aplicamos KBinsDiscretizer.

Tenemos que pasarle la cantidad de bins, encode y strategy.

Averiguar que significan estos parametros:

https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.KBinsDiscretizer.html

In [ ]:
from sklearn.preprocessing import KBinsDiscretizer
est = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy = 'uniform')

In [ ]:
# n_bins: cantidad de "cajas"(bines) en donde guardo mis datos
# encode: metodo usado para la transformación --- ‘onehot’ (nos devuelva sparse matrix) , ‘onehot-dense’ dense array , 'ordinal' te devuelve el  "bin identifier encoded"
# strategy:‘uniform’ todos los bines tinene el mismo ancho
#          :‘quantile’ todos los bines tienen la misma cant de datos
#          :'Values' in each bin have the same nearest center of a 1D k-means cluster.

En scikit learn siempre estuvimos aplicando el método fit y transform por separado. Scikit lern nos permite aplicar los dos en una linea con el método fit_transform:

In [ ]:
df_cont['discretized'] = est.fit_transform(df_cont[['X']]) # Acá en lugar de hacer reshape(-1, 1) utilizamos doble [[]]

In [ ]:
est.fit_transform(df_cont[['X']])

In [ ]:
df_cont

In [ ]:
df_cont.discretized.value_counts()

# Ejercicio

Vamos a levantar el dataset de la clase pasada (esta vez sin nulos) y transformar las variables categóricas.

Tienen que utilizar su criterio para decidir cuando conviene ordinal, one hot, etc.

Recuerden que las columnas del dataset son:


- id: Id del usuario 
- administrative: Número de veces que el usuario visito la sección "administrative"
- administrative_duration: Tiempo que el usuario paso en la sección administrative
- informational: Número de veces que el usuario visitó la sección "informational"
- informational_duration: Tiempo que el usuario paso en la sección informational
- productrelated: Número de veces que el usuario visitó la sección "products related"
- productrelated_duration: Tiempo que el usuario pasó en la sección 
- bouncerates: Porcentaje de visitantes que entran a la página e inmediatamente la dejan sin interactuar con la misma. Esta metrica solo se tiene en cuenta si es la primer página que se visitó del sitio web.
- exitrates: De la cantidad total de visitas a las páginas del sitio web, el porcentage de usuarios que lo abandonaron en esta página. Esto es, el  porcentaje de usuarios que su última visita al sitio fué en esta página.
- pagevalues: Este es el valor promedio del sitio web, indica la contribución que este sitio web hizo al visitante que llega a la página o sección de compra final.
-  specialday: Es una fecha especial o no (1 o 0)
- operatingsystems: Sistema operativo
- browser: Nombre del navegador
- region: Region geográfica del usuario
- traffictype: Tipo de tráfico web
- visitortype: Nuevo o uno que retorno al sitio
- Weekend: 1 si es fin de semana y 0 en otro caso
- revenue: 1 si el usuario hizo una compra y 0 en otro caso

In [ ]:
df = pd.read_csv('drive/MyDrive/ICARO/Curso DS/Clases/Clase 11/online-shoppers-intention-sin-nulos.csv')

In [ ]:
df.head()

Transformar las variables:

- Month
- Visitor type
- weekend

Con los métodos que aprendimos.

Discretizar:
- ExitRates
- BounceRates


Investigar:

- ¿Cómo puedo saber desde que valor hasta que valor van cada uno de los "bins" en KBinsDiscretizer? (buscar los atributos del discretizer en la documentación)
- ¿ Qué pasa si en lugar de usar encode="ordinal" uso encode=‘onehot’ o ‘onehot-dense’?
- ¿Cuál es la diferencia entre strategy=‘uniform’ y strategy=‘quantile’ ?